In [ ]:
from core.helpers.session_helper import SessionHelper
session = SessionHelper().session

In [ ]:
"""
************ CONFIGURATION - PLEASE TOUCH **************
Pipeline Builder configuration: creates configurations from variables specified here!!
This cell will be off in production as configurations will come from the configuration postgres DB.
"""
# config vars: this dataset
config_pharma = "sun" # the pharmaceutical company which owns {brand}
config_brand = "ilumya" # the brand this pipeline operates on
config_state = "enrich" # the state this transform runs in
config_name = "enrich_cancel_before_active" # the name of this transform, which is the name of this notebook without .ipynb

# input vars: dataset to fetch. Recall that a contract published to S3 has a key format branch/pharma/brand/state/name
# Note: this key is case sensitive!!
input_pharma = "sun"
input_brand = "ilumya"
input_state = "ingest"
input_name = "symphony_health_association_ingest_column_mapping"
input_branch = "sun-extract-validation" # if None, input_branch is automagically set to your working branch

In [ ]:
"""
************ SETUP - DON'T TOUCH **************
Populating config mocker based on config parameters...
"""
import core.helpers.pipeline_builder as builder

ids = builder.build(config_pharma, config_brand, config_state, config_name, session)
transform_id = ids[0]
run_id = ids[1]

In [ ]:
"""
************ SETUP - DON'T TOUCH **************
This section imports data from the configuration database
and should not need to be altered or otherwise messed with. 
~~These are not the droids you are looking for~~
"""
from core.constants import BRANCH_NAME, ENV_BUCKET
from core.helpers.session_helper import SessionHelper
from core.models.configuration import Transformation
from dataclasses import dataclass
from core.dataset_contract import DatasetContract

db_transform = session.query(Transformation).filter(Transformation.id == transform_id).one()

@dataclass
class DbTransform:
    id: int = db_transform.id ## the instance id of the transform in the config app
    name: str = db_transform.transformation_template.name ## the transform name in the config app
    state: str = db_transform.pipeline_state.pipeline_state_type.name ## the pipeline state, one of raw, ingest, master, enhance, enrich, metrics, dimensional
    branch:str = BRANCH_NAME ## the git branch for this execution 
    brand: str = db_transform.pipeline_state.pipeline.brand.name ## the pharma brand name
    pharmaceutical_company: str = db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name # the pharma company name
    publish_contract: DatasetContract = DatasetContract(branch=BRANCH_NAME,
                            state=db_transform.pipeline_state.pipeline_state_type.name,
                            parent=db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name,
                            child=db_transform.pipeline_state.pipeline.brand.name,
                            dataset=db_transform.transformation_template.name)


# CORE Cartridge Notebook::[transform name here]
![CORE Logo](assets/coreLogo.png) 

---
## Keep in Mind
Good Transforms Are...
- **singular in purpose:** good transforms do one and only one thing, and handle all known cases for that thing. 
- **repeatable:** transforms should be written in a way that they can be run against the same dataset an infinate number of times and get the same result every time. 
- **easy to read:** 99 times out of 100, readable, clear code that runs a little slower is more valuable than a mess that runs quickly. 
- **No 'magic numbers':** if a variable or function is not instantly obvious as to what it is or does, without context, maybe consider renaming it.

## Workflow - how to use this notebook to make science
#### Data Science
1. **Document your transform.** Fill out the _description_ cell below describing what it is this transform does; this will appear in the configuration application where Ops will create, configure and update pipelines. 
1. **Define your config object.** Fill out the _configuration_ cell below the commented-out guide to define the variables you want ops to set in the configuration application (these will populate here for every pipeline). 
2. **Build your transformation logic.** Use the transformation cell to do that magic that you do. 
![caution](assets/cautionTape.png)

### Configuration

In [ ]:
""" 
********* CONFIGURATION - PLEASE TOUCH ********* 
This section defines what you expect to get from the configuration application 
in a single "transform" object. Define the vars you need here, and comment inline to the right of them 
for all-in-one documentation. 
Engineering will build a production "transform" object for every pipeline that matches what you define here.

@@@ FORMAT OF THE DATA CLASS IS: @@@ 

<value_name>: <data_type> #<comment explaining what the value is to future us>

~~These ARE the droids you are looking for~~
"""

class Transform(DbTransform):
    '''
    YOUR properties go here!!
    Include your input dataset(s) here. Variable properties should be assigned to the exact name of
    the transformation as it appears in the Jupyter notebook filename.
    '''

In [ ]:
## Please place your value assignments for development here!!
## This cell will be turned off in production and Engineering will set to pull form the configuration application instead

transform = Transform()

### Description
What does this transformation do? be specific.

![what does your transform do](assets/what.gif)

Cancelled Before Active enrichment.
Assigns hierarchy values in cases where cancelled status is reported before first active shipment.  This is used as part of the TTFF enrichment.  See logic diagram below:

<img src='assets/cancel_before_active.png'>

### Transformation

In [ ]:
"""
************ FETCH DATA - TOUCH, BUT CAREFULLY **************
This cell will be turned off in production, as the input_contract will be handled by the pipeline.
"""

if not input_branch:
    input_branch = BRANCH_NAME
input_contract = DatasetContract(branch=input_branch, state=input_state, parent=input_pharma, child=input_brand, dataset=input_name)
run_filter = []
run_filter.append(dict(partition="__metadata_run_id", comparison="==", values=[4]))
# IF YOU HAVE PUBLISHED DATA MULTIPLE TIMES, uncomment the above line and change the int to the run_id to fetch.
# Otherwise, you will have duplicate values in your fetched dataset!
df = input_contract.fetch(filters=run_filter)

In [ ]:
df.shape

In [ ]:
### Use the variables above to execute your transformation. the final output needs to be a variable named final_dataframe

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

### DATA CLEANING: ADDRESS THIS SECTION BEFORE PIPELINE INTEGRATION

In [ ]:
if input_pharma == 'sun':
    trans_id = 'pharm_transaction_id'
    brand = 'medication'
    patient_id = 'msa_patient_id'
    pharmacy = 'pharm_code'
    status_date = 'status_date'
    referral_date = 'ref_date'
    status =  'status_code'
    substatus =  'sub_status'
    datetime = '%Y%m%d'
    active_substatus_code = 'SHIPMENT'
    cancel_status_code = 'CANCELLED'
    active_hierarchy = 'ACTIVE - SHIPMENT'

elif input_pharma == 'bi':
    trans_id = 'pmcTransactionId'
    brand = 'medication'
    patient_id = 'pmcPatientID'
    pharmacy = 'pharmName'
    status_date = 'statusDate'
    referral_date = 'refDate'
    status =  'statusCode'
    substatus =  'custStatusCode'
    datetime = '%Y-%b-%d'
    active_substatus_code = 'S01'
    cancel_status_code = 'CANCELLED'
    active_hierarchy = 'ACTIVE - S01'

In [ ]:
# CLEAN DATA - This step should not be necessary once transform is integrated into pipeline.
#    Extract and map relevant columns
#    Convert dates to datetime format
#    Extract brand from medication
#    Convert substatuses to uppercase
#    Populate null referral dates with the min(status_date) for that patient/pharmacy/brand.
    
def clean_data(cust_input_df, datetime, trans_id, brand, patient_id, pharmacy, status_date, referral_date, status, substatus):

    cust_df = (
        cust_input_df
        .loc[:,
             [trans_id,
              patient_id,
              pharmacy,
              brand,
              status_date,
              referral_date,
              status,
              substatus]
            ]
        .rename(
            columns={
                trans_id:'trans_id',
                patient_id:'patient_id',
                pharmacy:'pharmacy',
                brand:'brand',
                status_date:'status_date',
                referral_date:'referral_date',
                status:'status_code',
                substatus:'substatus_code'}
        )
        .assign(status_date = lambda x: (
            pd.to_datetime(
                x.status_date.str[:8].astype(str),
                format=datetime,
                errors='coerce'
            )
        ))
        .fillna(value={'referral_date':'min_status_date'})
        .assign(referral_date = lambda x: (
            pd.to_datetime(
                x.referral_date.str[:8].astype(str),
                format=datetime,
                errors='coerce'
            )
        ))
        .dropna()
        .assign(brand = lambda x: (
            x['brand'].apply(lambda x: x.split()[0].strip())
        ))
        .assign(status_code = lambda x: (
            x.status_code.str.upper()
        ))
        .assign(substatus_code = lambda x: (
            x.substatus_code.str.upper()
        ))
        .assign(min_status_date = lambda x: (
            x.groupby(['patient_id','pharmacy','brand'])['status_date']
            .transform(min)
        ))
        .drop(['min_status_date'],axis=1)
        .drop_duplicates()
        .sort_values(
            by=['patient_id', 'pharmacy', 'brand', 'status_date','status_code'],
            ascending=[True, True, True, True, False])
        .reset_index(drop=True)
    )

    return cust_df

In [ ]:
clean_df = clean_data(
    df,
    datetime,
    trans_id,
    brand,
    patient_id,
    pharmacy,
    status_date,
    referral_date,
    status,
    substatus
)

clean_df.head()

In [ ]:
clean_df.shape

### APPLY TRANSFORM LOGIC

In [ ]:
# Assign Patient Journey (pj_id) and Patient Journey Step (pj_step) identifiers
# (These IDs are used for calculation purposes only.  They will not be published)

pj_df = (
    clean_df
    .assign(pj_id = lambda x: (
        x.groupby(['patient_id','pharmacy','brand']).grouper.group_info[0]
    ))
    .assign(pj_step = lambda x: x.index)
)

pj_df.head()

In [ ]:
# Filter to only include patient journeys where:
#    a) Active Shipment status is reported
#    b) Cancelled status occurs prior to first active shipment

cancel_before_active_df = (
    pj_df
    .assign(active_step = lambda x: (
        np.where(
            x['substatus_code']==active_substatus_code,
            x['pj_step'],
            np.nan
    )))
    .assign(active_status_date = lambda x: (
        pd.to_datetime(np.where(
            x['substatus_code']==active_substatus_code,
            x['status_date'],
            pd.NaT
    ))))
    .assign(first_active_step = lambda x: (
        x.groupby(['pj_id'])['active_step']
        .transform(min)
    ))
    .assign(first_active_status_date = lambda x: (
        x.groupby(['pj_id'])['active_status_date']
        .transform(min)
    ))
    .drop(['active_step', 'active_status_date'], axis=1)
    .dropna(axis=0)
    .assign(active_cancel_diff = lambda x:(
        np.where(x['status_code'] == cancel_status_code,
                 (x['first_active_status_date'] - x['status_date']) / np.timedelta64(1, 'D'),
                 np.nan
                )
    ))
    .assign(active_cancel_diff = lambda x: (
        x.groupby(['pj_id'], sort=False)['active_cancel_diff']
        .transform(lambda x: x.bfill())
        ))
    .loc[lambda x: (
        x['pj_id'].isin(x
                        .loc[x['active_cancel_diff'] > 0]
                        .pj_id
                        .drop_duplicates()
                        .tolist()
                       )
    )]
)

cancel_before_active_df.head()

In [ ]:
# For each patient journey step, get the previous status and time spent in previous status.
# If it's the first step in the patient journey, show "no_prior_status"

prior_status_df = (
    cancel_before_active_df
    .assign(prior_status = lambda x:(
        x.groupby(['pj_id'])['status_code']
        .transform(lambda x: x.shift(1))
    ))
    .fillna(value={'prior_status':'no_prior_status'})
    .assign(prior_status_diff = lambda x: (
        x.groupby(['pj_id'])['status_date']
        .transform(lambda x: (x - x.shift(1))/np.timedelta64(1,'D'))
    ))
)

prior_status_df.head()

In [ ]:
# Apply logic to determine patient journey hierarchy. See logic diagram in transform description.
hierarchy_df = (
    prior_status_df
    .assign(hierarchy = lambda x:(
        np.where(
            x['pj_step'] >= x['first_active_step'],
            active_hierarchy,
            np.where(
                x['active_cancel_diff'] > 60,
                'REMOVE FROM TTFF',
                np.where(
                    (x['status_code'] != cancel_status_code),
                    x['status_code'] + ' - '+ x['substatus_code'], ## THIS WILL NEED TO BE CHANGED BEFORE PIPELINE INTEGRATION
                    np.where(
                        (x['status_code'] == cancel_status_code) & ((x['prior_status_diff'] > 60) | (x['prior_status'] == 'no_prior_status')),
                        'NO STATUS CLARITY',
                        np.where(
                            x['substatus_code'] == 'INSURANCE DENIED',
                            'BVPA',
                            None
                        )
                    )
                )
            )
        )
    ))
    .reset_index(drop=True)
    .assign(hierarchy = lambda x: (
        x.groupby(['pj_id'], sort=False)['hierarchy']
        .transform(lambda x: x.ffill())
        ))
)

hierarchy_df.head()

In [ ]:
final_dataframe = (
    hierarchy_df
    .drop(
        [
            'first_active_step',
            'first_active_status_date',
            'active_cancel_diff',
            'prior_status',
            'prior_status_diff',
            'pj_id',
            'pj_step'
        ],
        axis = 1)
)

final_dataframe.head()

In [ ]:
final_dataframe.shape

### Publish

In [ ]:
## that's it - just provide the final dataframe to the var final_dataframe and we take it from there
transform.publish_contract.publish(final_dataframe, run_id, session)
session.close()

In [ ]:
final_dataframe.info()